In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, TargetEncoder

In [37]:
train_df = pd.read_csv(r"datasets\dataset3_train.csv")
test_df = pd.read_csv(r"datasets\dataset3_test.csv")

train_df.info()
# test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Customer ID                  30000 non-null  object 
 1   Name                         30000 non-null  object 
 2   Gender                       29947 non-null  object 
 3   Age                          30000 non-null  int64  
 4   Income (USD)                 25424 non-null  float64
 5   Income Stability             28317 non-null  object 
 6   Profession                   30000 non-null  object 
 7   Type of Employment           22730 non-null  object 
 8   Location                     30000 non-null  object 
 9   Loan Amount Request (USD)    30000 non-null  float64
 10  Current Loan Expenses (USD)  29828 non-null  float64
 11  Expense Type 1               30000 non-null  object 
 12  Expense Type 2               30000 non-null  object 
 13  Dependents      

## Data Preprocessing

### Filtering unnecessary columns

In [38]:
train_df.drop(["Customer ID", "Name"],axis = 1, inplace=True)

### Handling null values

In [ ]:


## we observe null values present in following columns

null_columns = ["Gender", "Income (USD)", "Income Stability", "Type of Employment", "Current Loan Expenses (USD)", "Dependents", "Credit Score", "Has Active Credit Card", "Property Age", "Property Location"]

###Since loan sanction amount is the value to be predicted, columns with loansanctionamount null are removed
train_df.dropna(subset=["Loan Sanction Amount (USD)"],inplace=True)



## Encode categorical data
categorical = ["Gender", "Income Stability","Profession","Type of Employment","Location","Expense Type 1","Expense Type 2","Has Active Credit Card", "Property Location"]

## categories
categories = {
    "Gender":{'F', 'M'},
    "Income Stability":{'Low', 'High'},
    "Profession":{'Working', 'Pensioner', 'State servant', 'Commercial associate', 'Unemployed', 'Student', 'Businessman', 'Maternity leave'},
    "Type of Employment":{'Sales staff', 'High skill tech staff', 'Secretaries', 'Laborers', 'Managers', 'Cooking staff', 'Core staff', 'Drivers', 'Realty agents', 'Security staff', 'Accountants', 'Private service staff', 'Waiters/barmen staff', 'Medicine staff', 'Cleaning staff', 'Low-skill Laborers', 'HR staff', 'IT staff'},
    "Location":{'Semi-Urban', 'Rural', 'Urban'},
    "Expense Type 1":{'N', 'Y'},
    "Expense Type 2":{'N', 'Y'},
    "Has Active Credit Card":{'Unpossessed', 'Active', 'Inactive'},
    "Property Location":{'Rural', 'Urban', 'Semi-Urban'}
}

ordinal_encodings = {
    "Income Stability":['Low', 'High'],
    "Location":['Rural', 'Urban', 'Semi-Urban'],
    "Has Active Credit Card":['Unpossessed', 'Inactive', 'Active'],
    "Property Location":['Rural', 'Urban', 'Semi-Urban'],
    "Gender": ["M","F"],
    "Expense Type 1": ['Y', 'N'],
    "Expense Type 2": ['Y', 'N']
}

for column in ordinal_encodings:
    train_df[column] = OrdinalEncoder(categories=ordinal_encodings[column]).fit_transform(train_df[column])

for i in categorical:
    print(i, list(train_df[i].unique()))

#imputation
## fill median for Income (USD), Gender, Income Stability, Type of Employment, Current Loan Expenses (USD), Credit Score, Property Age
## fill 0 for Dependents
## fill Unposessed for Has Active Credit Card
## fill most frequent for Property Location

train_df["Dependents"] = train_df["Dependents"].fillna(0)
train_df["Has Active Credit Card"] = train_df["Has Active Credit Card"].fillna(0)
train_df["Property Location"] = train_df["Property Location"].fillna(1)
train_df["Gender"] = train_df["Gender"].fillna(1)
imputer = SimpleImputer(missing_values=np.nan, strategy="median")
train_df = imputer.fit_transform(train_df)


Gender ['F', 'M']
Income Stability ['Low', 'High', nan]
Profession ['Working', 'Pensioner', 'State servant', 'Commercial associate', 'Unemployed', 'Student', 'Businessman', 'Maternity leave']
Type of Employment ['Sales staff', nan, 'High skill tech staff', 'Secretaries', 'Laborers', 'Managers', 'Cooking staff', 'Core staff', 'Drivers', 'Realty agents', 'Security staff', 'Accountants', 'Private service staff', 'Waiters/barmen staff', 'Medicine staff', 'Cleaning staff', 'Low-skill Laborers', 'HR staff', 'IT staff']
Location ['Semi-Urban', 'Rural', 'Urban']
Expense Type 1 ['N', 'Y']
Expense Type 2 ['N', 'Y']
Has Active Credit Card ['Unpossessed', 'Active', 'Inactive']
Property Location ['Rural', 'Urban', 'Semi-Urban']


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'F'